<a href="https://colab.research.google.com/github/Jezandre/Python/blob/main/Projetos_Recomenda%C3%A7%C3%A3o_livros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introdução**

Sistema de recomendações estão cada vez mais comuns no nosso dia a dia, desde compras até qual filme pretendemos assistir na netflix. Tais sistemas possuem grandes aplicações no setor de vendas pois auxilia o consumidor a adquirir produtos baseado nas informações fornecidas.

Muita das vezes aquele produto que você nem pensou que precisava aparece ali no processo de compra simplesmente porque você adquiriu algo parecido. Muitos destes precessos utilizam  algoritimos de TF-IDF (Term Frequency-Inverse Document Frequency), que nada mais é que a avaliação de termos parecidos em uma descrição e comparação com outros produtos.

**Objetivo**

O presente artigo é apenas uma introdução a esse mundo, demosntrar e descrever pois faz parte do meu processo de aprendizagem. Uma maneira de enxergar um pouco como funciona um sistema de recomendações baseadas na descrição.

Os dataset utilizados estão disponíveis na plataforma kaggle no  e são informações de livros. O objetivo é indicar livros baseados nas descrições fornecidas na descrição.

Utilizei o seguinte vídeo como guia para a elaboração https://www.youtube.com/watch?v=ijtxuF_5kEU. Vale a pena ressaltar que o processo de criação de recomendações baseados na descrição é apenas um passo, outros passos precisam estar associados para que as recomendações possam ser ainda mais precisas.

**Passo a Passo**

Para iniciar primeiro precisamos fazer o download dos datasets disponíveis no link: https://www.kaggle.com/datasets/jealousleopard/goodreadsbooks

A implementação do código utilizei o google Colab pois é uma IDE bem simples de trabalhar e possui muitas bibliotecas do Python já instalados.

Como código de programação utilizei o Python 3.10.12

In [ ]:
!python --version

Python 3.10.12


**Importando Bibliotecas**

As bibliotecas utilizadas serão pandas para criação de dataframes, numpy para tratamento dos dados. TfidfVectorizer para vetorização de texto que é uma das principais etapas do processamento de linguagem natural. linear_kernel que será utilizado para o cálculo de produto escalar dos vetores criados.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



**Importando datasets**

No googlecolab você pode utilizar a pasta no canto esquerda para fazer o upload dos seus arquivos e utilizar o endereço a partir da localização dele para importar os seus dataframes. A importação é feita utilizando a biblioteca pandas seguido dos seguintes comandos e localização do arquivo.



In [ ]:
df_books = pd.read_csv('/content/goodreads_data.csv')


**Analise do dataset**

É importante sempre que importar um arquivo para o dataframe utilizar a função head para visualizar as colunas.

In [ ]:
df_books.head()

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...


Assim a parte que precisamos do dataset para utilizar o sistema de recomendação é a coluna "*overview*" pois é nela que temos os dados de sinopse que utilizaremos para criar o nosso sistema de recomendação.

In [ ]:
df_books['Description']

0       The unforgettable novel of a childhood in a sl...
1       Harry Potter thinks he is an ordinary boy - un...
2       Since its immediate success in 1813, Pride and...
3       Discovered in the attic in which she spent the...
4       Librarian's note: There is an Alternate Cover ...
                              ...                        
9995    How far would you go? If human society was gen...
9996    Jeth Cavanaugh is searching for a new life alo...
9997    This dark fable tells the story of four Englis...
9998    For Adriana Monroe life couldn’t get any bette...
9999    After demands of thousands of fans in various ...
Name: Description, Length: 10000, dtype: object

**Tratamento dos dados**

Como qualquer dataset que você for utilizar para análise ou ciencia de dados o tratamento de dados é uma das etapas mais importantes, é preciso garantir que as informações estejam adequadas para a utilização de algorítiomos de AI.

Para o nosso estudo aqui utilizaremos a remoção de stopwords. O que seria isso? Stop words são palavras de ligação que muita das fezes não tem uma relevância grande para a informação neste caso. Seriam as preposições, artigos, etc.

Nesse caso TfidfVectorizer possui uma função para a remoção dessas palavras em vários idiomas (Não vou saber dizer ao certo quais idiomas).

No caso do nosso dataset estamos utilizando a base em inglês então o parâmetro a ser utilizado é "*english*".

Precisamos tambem remover os valores nulo do nosso data set e para isso utilizamos o fillna("") que vai substituir estes valores por  vazio.

E por fim para aplicarmos ao nosso dataset a remoção de *stopwords* utilizamos a função fit_transform, assim teremos uma base apenas com as palavras mais relevantes para nosso contexto.



In [ ]:
tfidf = TfidfVectorizer(stop_words="english")
df_books['Description'] = df_books['Description'].fillna("")
tfidf_matrix = tfidf.fit_transform(df_books['Description'])

 **Calculando o produto vetorial**

 Em seguida utiliza-se o linear_kernel para calcular o produto vetorial da seguinte maneira. Esse cáculo é fundamental pois é a partir dele que vamos descobrir a importancia relativa de uma palavra dentro da descrição do filme. O processo transforma valores textuais em números de forma que a máquina possa "compreender" melhor e trazer informções mais acertivas

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

**Indices**

Em seguida é preciso remover os ídices duplicados do dataset. Índices duplicados podem causar erros de relacionamento e trazer informações erradas.

In [ ]:
indices = pd.Series(df_books.index, index=df_books['Book']).drop_duplicates()
indices

Book
To Kill a Mockingbird                                             0
Harry Potter and the Philosopher’s Stone (Harry Potter, #1)       1
Pride and Prejudice                                               2
The Diary of a Young Girl                                         3
Animal Farm                                                       4
                                                               ... 
Breeders (Breeders Trilogy, #1)                                9995
Dynamo                                                         9996
The Republic of Trees                                          9997
Waking Up (Healing Hearts, #1)                                 9998
Bits and Pieces: Tales and Sonnets                             9999
Length: 10000, dtype: int64

In [ ]:
indices['Pride and Prejudice']

2

**Criando função de recomendação**

A função criada a seguir recebe dois parâmetros, o "*title*" título que é o ítem para o qual você quer saber quais files serão recomendados e "*consine_sim*" que é um parametro opcional.

Quando temos esta expressão nos argumentos da função:

"cosine_sim = cosine_sim"

significa que se você já calculou ele anteriormente o valor pode ser utilizado.

O idx vai obter o índice do título a ser pesquisado no dataframe

idx = indice[title]

sim_scores avalia a similaridade do filme em relação ao filme que foi selecionado e cria uma lista com os índices que posteriormente será relacionado com os dados do filme no dataframe através da função iloc. E pronto temos um sistema simples de recomendação baseado na descrição da sinopse.



In [ ]:
def get_recommendations(cosine_sim = cosine_sim):
  pd.set_option('display.max_rows', None)
  print("Available Books:")
  print(df_books['Book'])

# Get user input
  title = str(input("Enter the title of the book you want: "))

# Check if the selected book is in the DataFrame
  if title in df_books['Book'].values:
    print(f"You've selected: {title}")
    idx = indices[title]
    sim_scores = enumerate(cosine_sim[idx])
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    cosine_sim[idx]
    sim_index = [i[0] for i in sim_scores]
    print(sim_index)
    print(df_books['Book'].iloc[sim_index])

    # Now you can proceed with your recommendation system using the selected book
  else:
    print("Invalid selection. Please choose a book from the list.")

get_recommendations()

A saída de streaming foi truncada nas últimas 5000 linhas.
5015                   The Rosie Effect (Don Tillman, #2)
5016                                    Tikki Tikki Tembo
5017    The Snowball: Warren Buffett and the Business ...
5018                                  Motherless Brooklyn
5019            Evolution: Vol.1 A Short Story Collection
5020         From Doon With Death (Inspector Wexford, #1)
5021    The Happiness Advantage: The Seven Principles ...
5022    The Naughtiest Girl in the School (The Naughti...
5023               La resistencia (Memorias de Idhún, #1)
5024                               Many Sparrows: A Novel
5025                                        Dying to Give
5026                                            Yoga Time
5027                            Carry On (Simon Snow, #1)
5028                                           Chasm City
5029             Running with the Demon (Word & Void, #1)
5030                              Bitten (Otherworld, #1)
5031         


**Conclusão**

Como podemos ver o sistema desenvolvido em questão é uma das mais simples aplicações para recomendação. Existem diversos algorítmos bem mais robustos que podem trazer muito mais resultados. Porém explorar os conceitos básicos são primordiais para o desenvolvimento de aplicações mais robustas. Quem nunca se deparou nos estudos e no trabalho se lembrando de um conceito fundamental que pode ser aplicado na prática não é mesmo.

Sujestões de aplicação correções estejam livres em colocar nos comentários.

